In [1]:
%%capture
!pip install transformers

In [39]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import csv
from sklearn.utils import shuffle
import tensorflow as tf
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel , AutoModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import unicodedata
import string
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import joblib

In [3]:
device = torch.device('cpu') 
data_path   = '/Users/catalina.chirita/Desktop/Disertatie/RomanianSatireDataset/'
train_path = data_path + "train.csv"
test_path = data_path + "test.csv"
val_path = data_path + "validation.csv"

In [4]:
def createDataset(data_path, path_texts):
    dataset = pd.read_csv(path_texts, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    dataset['label']= dataset['label'].apply(lambda x:'satire' if x==1 else 'nonSatire')
    dataset.dropna(subset=['content'],inplace = True)
    dataset.reset_index(drop=True, inplace=True)
    dataset.drop(columns = ['title','index'], axis =1 , inplace = True)
    return dataset

pd_train = createDataset(data_path,train_path)
pd_test = createDataset(data_path,test_path)
pd_val = createDataset(data_path,val_path)

In [7]:
pd_train['content'][1]

'În cadrul unei întâlnirii a $NE$ de $NE$ din $NE$ $NE$ $NE$ şefii marilor agenţii de informaţii americane, printre care $NE$ $NE$ şi $NE$ au emis o comunicare comună prin care avertizează cu privire la riscurile folosirii dispozitivelor mobile produse de companiile chinezeşti, informează news . ro . Şeful $NE$ $NE$ $NE$ a spus că guvernul american este foarte îngrijorat din cauza riscurilor de securitate pe care răspândirea produselor unor companii fidele unui regim politic care nu împărtăşeşte aceleaşi valori democratice le poate reprezenta . Conform directorului $NE$ aceste companii au capacitatea de a fura şi modifica informaţiile în mod fraudulos, creând terenul propice pentru spionaj . În prim planul acestui scandal este $NE$ companie a cărei divizie de smartphone - uri a crescut foarte mult în ultimii ani . Şefii agenţiilor americane avertizează că fondatorul $NE$ este un fost membru al unei divizii a guvernului chinez, numite $NE$ de $NE$ a $NE$ $NE$ . În 2014, celor de la $NE$

In [6]:
pd_train = pd_train.sample(n=8000) 
pd_train.reset_index(drop=True, inplace=True)
pd_test.reset_index(drop=True, inplace=True)
print(pd_train)
print(pd_test)

                                                content      label
0      „Adevărul, dragă $NE$ – mi-a spus azi-noapte ...     satire
1     Gen $NE$ $NE$ $NE$ $NE$ (manelistă virgină): $...     satire
2     Din nefericire, nu există un tribunal al bunul...     satire
3     Site-ul $NE$ de $NE$ din $NE$ a căzut la cîtev...     satire
4     Îl știți pe $NE$ luzărul de vreo 30 de ani, vî...     satire
...                                                 ...        ...
7995  V$NE$ . Meteorologii au emis duminică o inform...  nonSatire
7996  * $NE$ o jumătate de săptămână, $NE$ $NE$ spum...     satire
7997   În ultimul timp, un cetăţean pe care-l cheamă...     satire
7998  Aceste informatii sunt vitale, daca omenirea v...  nonSatire
7999  Podcasturile sunt pe val, iar la $NE$ a aparut...  nonSatire

[8000 rows x 2 columns]
                                                content      label
0      Ăsta dascăl! $NE$ de conștiinciozitate la o ș...     satire
1     Sloganul lor, $NE$ sare? $NE$ s

In [7]:
satire = 0
nonsatire=0
for array in range(len(pd_train["content"])):
    if pd_train["label"][array] == "satire":
        satire= satire+1
    else:
        nonsatire=nonsatire+1


In [8]:
print(satire)
print(nonsatire)

3980
4020


In [9]:
for array in range(len(pd_train["content"])):
    pd_train["content"][array]=unicodedata.normalize('NFD', pd_train["content"][array]).encode('ascii', 'ignore').decode("utf-8")
     
for array in range(len(pd_test["content"])):
    pd_test["content"][array]=unicodedata.normalize('NFD', pd_test["content"][array]).encode('ascii', 'ignore').decode("utf-8")
    
for array in range(len(pd_val["content"])):
    pd_val["content"][array]=unicodedata.normalize('NFD', pd_val["content"][array]).encode('ascii', 'ignore').decode("utf-8")    
    


In [10]:
pd_train['content'][0]

' Adevarul, draga $NE$  mi-a spus azi-noapte presedintele, dupa ce s-a uitat la reportajul de la $NE$ 3, in care $NE$ $NE$ $NE$ $NE$ $NE$ inscria cos dupa cos pe linga $NE$ $NE$ , e ca cel mai mult imi place daca nu beau. Tu nu vezi ca atunci cind sint treaz nu-mi rezista nimeni? $NE$ nu sint bon viveur, ca $NE$ eu sint mauvais viveur, daca pot pentru ca sa ma exprim asa, si uite ca pot! $NE$ drept, nici figura nu ma ajuta prea mult: de pilda, dimineata cind ma trezesc si ma uit in oglinda, am o fata de mi se pare c-am baut toata noaptea. Si cind colo, eu citisem $NE$ $NE$ nici sa citesti prea mult nu e bine. Ceva din $NE$ meu ma-mpiedica sa fiu, cel putin la masa, prea cultural. Ce, crezi ca eu n-as putea cita din $NE$ din $NE$ sau din $NE$ asta pe care-l citeaza toti? $NE$ e bine nici sa scrii si cu atit mai rau e sa citezi, fiindca te dezvalui. Vorba ceea: $NE$ ce citezi ca sa-ti spun cine esti. $NE$ dom presedinte, $NE$ l-ati terminat?  am plusat eu. $NE$ l-am terminat, fiindca mi-

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords

pd_train['content']= pd_train['content'].apply(lambda x: word_tokenize(x))
pd_test['content']= pd_test['content'].apply(lambda x: word_tokenize(x))

final_stopwords_list = stopwords.words('romanian')
final_stopwords_list.append('«')
final_stopwords_list.append('»')
final_stopwords_list.append('’')
final_stopwords_list.append('-')
final_stopwords_list.append('$')
final_stopwords_list.append('NE')
final_stopwords_list.append('(')
final_stopwords_list.append(')')
def remove_stopwords(text):
    output= [i for i in text if i not in final_stopwords_list]
    return output
def remove_digit(text):
    output= [c for c in text if not c.isdigit()]
    return output

pd_train['content']= pd_train['content'].apply(lambda x:remove_stopwords(x))
pd_train['content']= pd_train['content'].apply(lambda x:remove_digit(x))

pd_test['content']= pd_test['content'].apply(lambda x:remove_stopwords(x))
pd_test['content']= pd_test['content'].apply(lambda x:remove_digit(x))

for array in range(len(pd_train["content"])):
    pd_train["content"][array]=' '.join(pd_train["content"][array]) 
    
for array in range(len(pd_test["content"])):
    pd_test["content"][array]=' '.join(pd_test["content"][array]) 

In [9]:
pd_test['content'][0]

' Ăsta dascăl! $NE$ de conștiinciozitate la o școală din $NE$ unde un profesor a alergat 8 km în 22 de minute, până la casa elevului care mesteca gumă la ora online, elev pe care l-a urechiat, așa cum prevede regulamentul școlilor românești. Proful a trebuit să se întoarcă repede la oră, pentru că un alt elev îi șoptise altuia la test, așa că a trebuit să se ducă să-l urecheze și pe el. $NE$ păcate unul din părinții băiatului a încercat să se împotrivească, așa că i-am dat un cap în gură. Ai o problemă, o ridici la ședința cu părinții, imbecilule, mie nu-mi spui cum să educ copiii”, spune proful. Acesta se plânge că elevii sunt cam neastâmpărați și în timpul orelor aleargă chiar și un semimaraton ca să-i pedepsească. Elevii s-au adaptat rapid și folosesc $NE$ ca să știe pe unde a ajuns proful și în ultimul moment fug și ei din casă, înainte să vină peste ei și să-i urecheze. Ei spun că nu e normal să fie urechiați pentru mestecat gumă, din moment ce proful face orele online în chiloți 

In [12]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
pd_train['content']= pd_train['content'].apply(lambda x:remove_punctuation(x))
pd_train['content']= pd_train['content'].apply(lambda x: x.lower())

pd_test['content']= pd_test['content'].apply(lambda x:remove_punctuation(x))
pd_test['content']= pd_test['content'].apply(lambda x: x.lower())

In [13]:
pd_test['content'][0]

'asta dascal  constiinciozitate scoala profesor alergat km minute  pana casa elevului mesteca guma ora online  elev la urechiat  prevede regulamentul scolilor romanesti  proful trebuit intoarca repede ora  elev soptise altuia test  trebuit duca sal urecheze  pacate parintii baiatului incercat impotriveasca  iam cap gura  ai problema  ridici sedinta parintii  imbecilule  numi spui educ copiii  spune proful  acesta plange elevii neastamparati timpul orelor alearga semimaraton sai pedepseasca  elevii sau adaptat rapid folosesc stie ajuns proful ultimul moment fug casa  vina sai urecheze  ei spun normal urechiati mestecat guma  moment proful orele online chiloti paharul vodca langa '

In [14]:
pd_train.to_pickle("romaniandataset.pkl")
pd_test.to_pickle("testromaniandataset.pkl")

In [15]:
np.random.seed(112)
df_train, df_val, df_test = np.split(pd_train.sample(frac=1, random_state=42), 
                                     [int(.8*len(pd_train)), int(.9*len(pd_train))])

print(len(df_train),len(df_val), len(df_test))

6400 800 800


In [40]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        #embedding vectors of all of the tokens in a sequence + embedding vector of [CLS] token
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer , pooled_output
        

In [41]:
from transformers import AutoTokenizer, AutoModel
import torch
# load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")

labels = {'nonSatire':0,
          'satire':1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['label']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y
    

In [16]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [42]:
from sklearn.ensemble import VotingClassifier
import pandas as pd
    
def evaluate2models2(model,test_dataset):

    test = Dataset(test_dataset)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    dataframe_val = []
    dataframe_wght = []
    dataframe_satire=[]
    dataframe_label = []
    dataframe_cls =[]
    pooled_output_dat = []
    with torch.no_grad():

    
        for test_input, test_label in test_dataloader:

                test_label = test_label.to(device)
                mask = test_input['attention_mask'].to(device)
                input_id = test_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                pooled_output = output[1]
                print("output", output[0])
                print("pooled-output", pooled_output)
                acc = (output[0].argmax(dim=1) == test_label).sum().item()
                if acc == 1:
                    if output[0].argmax(dim=1)[0] == 1:
                        dataframe_satire.append('satire')
                        print('satire')
                    else:
                        dataframe_satire.append('nonSatire')
                        print('nonSatire')
                else:
                    dataframe_satire.append('error')
                    print('error of preddiction')
                    
                pooled_output = pooled_output.tolist()
                pooled_output_dat.append(pooled_output)
                output = output[0].tolist()  
                test_label = test_label.tolist()
                dataframe_val.append(output[0])
                dataframe_wght.append(float(0.60))
                dataframe_label.append(test_label[0])
                total_acc_test += acc
    
        model_dataset = {'Value':dataframe_val ,
        'Weight': dataframe_wght,
        'Label': dataframe_label,
        'Type': dataframe_satire}
        
        model_dataset = pd.DataFrame(model_dataset)
        model_dataset.to_pickle("dataframero.pkl")
        
        model_d = {'Pooled output romanian':pooled_output_dat}
 
        model_dataset2 = pd.DataFrame(model_d)
        model_dataset2.to_pickle("dataframero_pldout.pkl")
        
        print("Model1 DF", model_dataset)
        print(f'Test Accuracy model: {total_acc_test / len(test_dataset): .3f}')

In [20]:
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at dumitrescustefan/bert-base-romanian-cased-v1 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/3200 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 3200/3200 [6:47:54<00:00,  7.65s/it]  


Epochs: 1 | Train Loss:  0.097 | Train Accuracy:  0.927 | Val Loss:  0.021 | Val Accuracy:  0.988


100%|██████████| 3200/3200 [19:42:24<00:00, 22.17s/it]        


...

Epochs: 3 | Train Loss:  0.004 | Train Accuracy:  0.999 | Val Loss:  0.010 | Val Accuracy:  0.994


100%|██████████| 3200/3200 [16:04:16<00:00, 18.08s/it]       


Epochs: 4 | Train Loss:  0.002 | Train Accuracy:  1.000 | Val Loss:  0.012 | Val Accuracy:  0.995


100%|██████████| 3200/3200 [47:08:46<00:00, 53.04s/it]        


Epochs: 5 | Train Loss:  0.001 | Train Accuracy:  1.000 | Val Loss:  0.011 | Val Accuracy:  0.995


In [21]:

filename = 'finalized_model_rom.sav'
joblib.dump(model, filename)

['finalized_model_rom.sav']

In [8]:
device = torch.device('cpu') 
data_path   = '/Users/catalina.chirita/Desktop/Disertatie/RomanianSatireDataset/'
test_path = data_path + "test.csv"
pd_test = createDataset(data_path,test_path)

In [43]:
filename = 'finalized_model_rom.sav'
loaded_model = joblib.load(filename)

pd_test = pd.read_pickle("testromaniandataset.pkl")
evaluate2models2(loaded_model, pd_test)

In [26]:
pd_train = pd.read_pickle("romaniandataset.pkl")
pd_train_eng = pd.read_pickle("romaniantoenglish.pkl")

pd_test = pd.read_pickle("testromaniandataset.pkl")
pd_test_eng = pd.read_pickle("testromaniantoenglish.pkl")

In [27]:
pd_train

,content,label
0,adevarul draga mia spus azinoapte presedintel...,satire
1,gen manelista virgina virgina de ani iubit ...,satire
2,din nefericire exista tribunal bunuluisimt n...,satire
3,siteul cazut minute anuntat cistigator alegeri...,satire
4,il stiti luzarul ani vinzator intrun fastdrin...,satire
...,...,...
7995,v meteorologii emis duminica informare vizand...,nonSatire
7996,jumatate saptamana spumega plin apuca ameni...,satire
7997,in ultimul timp cetatean carel cheama comente...,satire
7998,aceste informatii vitale omenirea vrea ajunga...,nonSatire


In [28]:
pd_train_eng

,content,label
0,[read the previous episode become a writer I h...,satire
1,[is extended about years according to the stud...,nonSatire
2,[British daily written on Mondays material ded...,nonSatire
3,[kind lucky women numbers among women left pre...,satire
4,[Roman Communism napa unaffirmed breads preven...,satire
...,...,...
7995,[if along the first two seasons of appeared th...,satire
7996,[the year of the possibilities and the impossi...,satire
7997,[car brand ceases to sell models markets such ...,nonSatire
7998,[1100 p.m. take the parade stand the rostrum w...,nonSatire


In [29]:
pd_test_eng

,content,label
0,[this dassacal awareness school teacher run km...,satire
1,[The slogan salt salt transposed reality gasme...,satire
2,[we have confirmation of calculated science le...,satire
3,[already appeared cracks team announced althou...,satire
4,[Our great champion photographed recently bath...,satire
...,...,...
9790,[update 2030 verification procedures astronaut...,nonSatire
9791,[the balance of the coronavirus infections exc...,nonSatire
9792,[update time 1040 reports deaths cause coronav...,nonSatire
9793,[authorities report deaths cause coronavirus t...,nonSatire


In [30]:
pd_test

,content,label
0,asta dascal constiinciozitate scoala profesor...,satire
1,sloganul sare sare transpus realitate gazm...,satire
2,avem confirmarea stiinta calculat duce viata ...,satire
3,deja aparut fisuri echipa anuntata desi sigur...,satire
4,marea noastra campioana fotografiata recent fa...,satire
...,...,...
9790,update 2030 verificari proceduri astronautii...,nonSatire
9791,bilantul infectiilor coronavirus depasit pragu...,nonSatire
9792,update ora 1040 raporteaza decese cauza corona...,nonSatire
9793,autoritatile raporteaza decese cauza coronavir...,nonSatire


In [31]:
pd_test["content"][0]

'asta dascal  constiinciozitate scoala profesor alergat km minute  pana casa elevului mesteca guma ora online  elev la urechiat  prevede regulamentul scolilor romanesti  proful trebuit intoarca repede ora  elev soptise altuia test  trebuit duca sal urecheze  pacate parintii baiatului incercat impotriveasca  iam cap gura  ai problema  ridici sedinta parintii  imbecilule  numi spui educ copiii  spune proful  acesta plange elevii neastamparati timpul orelor alearga semimaraton sai pedepseasca  elevii sau adaptat rapid folosesc stie ajuns proful ultimul moment fug casa  vina sai urecheze  ei spun normal urechiati mestecat guma  moment proful orele online chiloti paharul vodca langa '

In [32]:
pd_test_eng["content"][0]

["this dassacal awareness school teacher run km minutes to student's house chew gum time online student at the ear provides the regulation of Romanian schools the teacher had to quickly return the student's time sown another test had to take salt earings sins parents of the boy tried against the mouth of the high problem meeting stupid parents call you educ children tell the teacher this crys unmatched students during the hours run his semimaraton punishes students or adapts"]